## Model Explainability using Class Activation Maps

In [ ]:
!pip install grad-cam

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.model_targets import BinaryClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

In [ ]:
#download model file from S3
def download_data_from_s3(s3_uri):
    S3Downloader.download(s3_uri=s3_uri,local_path='<define path>',sagemaker_session=sagemaker_session)

In [ ]:
#Load model
model = torch.load(local_path)

In [ ]:
dict(model.named_modules())

In [ ]:
img = Image.open("test_images/ID_7a097bdf4.jpg")


target_layers = [model.features.denseblock4.denselayer16.conv1]
#target_layers = [model.features.denseblock4.denselayer16.norm2]

convert_tensor = transforms.Compose([transforms.Grayscale(num_output_channels=3),transforms.ToTensor()])
input_tensor = convert_tensor(img).unsqueeze(0)
#input_tensor = # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

In [ ]:
input_tensor.shape

In [ ]:
rgb_img = np.float32(img) / 255
rgb_img = np.stack((rgb_img,)*3, axis=-1)

In [ ]:
rgb_img.shape

In [ ]:
# Construct the CAM object once, and then re-use it on many images:
cam = GradCAMPlusPlus(model=model, target_layers=target_layers)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers, use_cuda=args.use_cuda) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.

targets = [ClassifierOutputTarget(1)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img,grayscale_cam, use_rgb=True)

In [ ]:
Image.fromarray(visualization)

In [ ]:
Image.fromarray(visualization).save("<Image_Created_from_CAM>")